In [ ]:
from ansatz import Ansatz
from utils import circuit_stats, find_original_param, get_subgroup_unitaries, pennylane_to_qiskit
import json
from twirlator.symmetry_groups import create_symmetric_group
from twirlator.generators import get_ansatz_generators
from ansatz import Ansatz
from twirlator.twirling import apply_twirling_to_generators
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp

all_results = {}
n_qubits = 5
S = create_symmetric_group(n_qubits)

with open(f"groups/subgroups_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = get_subgroup_unitaries(subgroups, S)

for depth in range(1, 6):
    print(f"  Depth {depth}...")
    for ansatz_id in range(1, 20):
        print(f"    Ansatz {ansatz_id}...")
        result_entry = {}
        super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
        circuit, params = super_ansatz.get_QNode()
        qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
        qiskit_circuit.remove_final_measurements()
        result_entry['original_stats'] = circuit_stats(qiskit_circuit)
        ansatz_generators = get_ansatz_generators(super_ansatz.get_ansatz())
        twirled_stats = {}

        for k in subgroup_unitaries:
            twirled_stats[k] = []
            for elem in subgroup_unitaries[k]:
                unitaries = elem["unitaries"]
                twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, n_qubits)
                twirled_circuit = QuantumCircuit(n_qubits)
                for i, (gen_matrix, op_wires, op_name, theta, is_parametric) in enumerate(ansatz_generators):
                    twirled_elem = twirled_generators[i]
                    if twirled_elem["gate_name"] == op_name and twirled_elem["wires"] == op_wires:
                        H = twirled_elem['averaged']
                        if is_parametric:
                            param = find_original_param(qiskit_circuit, ansatz_generators, i, op_name, op_wires, theta)
                        else:
                            param = theta
                        pauli_op = SparsePauliOp.from_operator(H)
                        evo_gate = PauliEvolutionGate(pauli_op, time=param)
                        twirled_circuit.append(evo_gate, range(n_qubits))
                    else:
                        raise ValueError(f"Twirled generator for {op_name} on wires {op_wires} not found when {twirled_elem['gate_name']} and {twirled_elem['wires']}")
                        
                transpiled_circuit = transpile(
                    twirled_circuit,
                    basis_gates=['rz', 'sx', 'cx', 'h', 'x', 'y', 'z', 'rx', 'ry', 'cz', 'rxx', 'rzz'],
                    optimization_level=3
                )
                twirled_circuit_stats = circuit_stats(transpiled_circuit)
                twirled_stats[k].append(twirled_circuit_stats)

        result_entry['twirled_stats'] = twirled_stats
        all_results.setdefault(str(n_qubits), {}).setdefault(str(depth), {})[str(ansatz_id)] = result_entry

with open(f"results/all_circuit_stats_{n_qubits}.json", "w") as f:
    json.dump(all_results, f, indent=4)


  Depth 1...
    Ansatz 1...
    Ansatz 2...
    Ansatz 3...
    Ansatz 4...
    Ansatz 5...
    Ansatz 6...
    Ansatz 7...
    Ansatz 8...
    Ansatz 9...
    Ansatz 10...
    Ansatz 11...
    Ansatz 12...
    Ansatz 13...
    Ansatz 14...
    Ansatz 15...
    Ansatz 16...
    Ansatz 17...
    Ansatz 18...
    Ansatz 19...
  Depth 2...
    Ansatz 1...
    Ansatz 2...
    Ansatz 3...
    Ansatz 4...
    Ansatz 5...
    Ansatz 6...
    Ansatz 7...
    Ansatz 8...
    Ansatz 9...
    Ansatz 10...
    Ansatz 11...
    Ansatz 12...
    Ansatz 13...
    Ansatz 14...
    Ansatz 15...
    Ansatz 16...
    Ansatz 17...
    Ansatz 18...
    Ansatz 19...
  Depth 3...
    Ansatz 1...
    Ansatz 2...
    Ansatz 3...
    Ansatz 4...
    Ansatz 5...
    Ansatz 6...
    Ansatz 7...
    Ansatz 8...
    Ansatz 9...
    Ansatz 10...
    Ansatz 11...
    Ansatz 12...
    Ansatz 13...
    Ansatz 14...
    Ansatz 15...
    Ansatz 16...
    Ansatz 17...
    Ansatz 18...
    Ansatz 19...
  Depth 4...
    An